# Imports

In [1]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r
!conda install -y python=3.10.13
!git clone https://github.com/bytedance/LatentSync.git
%cd LatentSync
!pip install torch==2.5.1 torchvision==0.20.1 --extra-index-url https://download.pytorch.org/whl/cu121
!uv pip install \
diffusers==0.32.2 \
transformers==4.48.0 \
decord==0.6.0 \
accelerate==0.26.1 \
einops==0.7.0 \
omegaconf==2.3.0 \
opencv-python==4.9.0.80 \
mediapipe==0.10.11 \
numpy==1.26.4 \
python_speech_features==0.6 \
librosa==0.10.1 \
scenedetect==0.6.1 \
ffmpeg-python==0.2.0 \
imageio==2.31.1 \
imageio-ffmpeg==0.5.1 \
lpips==0.1.4 \
face-alignment==1.4.1 \
gradio==5.24.0 \
huggingface-hub==0.30.2 \
protobuf==3.20.3 \
kornia==0.8.0 \
insightface==0.7.3 \
onnxruntime-gpu==1.21.0 \
av \
pydub==0.25.1 \
audioread==3.1.0 \
soundfile==0.13.1 \
DeepCache==0.1.1

!pip install numpy==1.26.4
!pip install --force-reinstall --no-cache-dir scikit-image
!grep "CONFIG_PATH" /kaggle/working/LatentSync/gradio_app.py

!apt-get install -y libgl1
!huggingface-cli download ByteDance/LatentSync-1.5 whisper/tiny.pt --local-dir /kaggle/working/LatentSync/checkpoints
!huggingface-cli download ByteDance/LatentSync-1.5 latentsync_unet.pt --local-dir /kaggle/working/LatentSync/checkpoints
import os
os.environ['MPLBACKEND'] = 'Agg'
import sys
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
import os
import torch
import gc
import subprocess
import time
import glob
sys.path.insert(0, '/kaggle/working/LatentSync')
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['ALSA_CARD'] = 'null'
!pip install "numpy<2" --force-reinstall
!sed -i 's|configs/unet/stage2_512.yaml|configs/unet/stage2.yaml|g' /kaggle/working/LatentSync/gradio_app.py
!grep "CONFIG_PATH" /kaggle/working/LatentSync/gradio_app.py

--2025-11-27 08:51:28--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 157891003 (151M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh.2’

Miniconda3-latest-L 100%[===================>] 150.58M   277MB/s    in 0.5s    

2025-11-27 08:51:29 (277 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh.2’ saved [157891003/157891003]

PREFIX=/usr/local
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, pl

error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evalu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 30.7 MB/s  0:00:00m0:00:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.24.0 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
CONFIG_PATH = Path("configs/unet/stage2.yaml")
    config = OmegaConf.load(CONFIG_PATH)


# Start of inference

In [ ]:
start = time.time()
WORK_DIR = "/kaggle/working"
os.makedirs(WORK_DIR, exist_ok=True)

input_path = "/kaggle/input/neilafaw"
search_pattern = os.path.join(input_path, "audio_*.wav")
files = glob.glob(search_pattern)

for i in range(len(files)):
    output_path = f"{WORK_DIR}/processed_part{i}.mp4"
    
    subprocess.run([
        "python", "-m", "scripts.inference",
        "--unet_config_path", "configs/unet/stage2.yaml",
        "--inference_ckpt_path", "checkpoints/latentsync_unet.pt",
        "--inference_steps", "30",
        "--guidance_scale", "1.5",
        "--enable_deepcache",
        "--video_path", f"{input_path}/video_{i}.mp4",
        "--audio_path", f"{input_path}/audio_{i}.wav",
        "--video_out_path", output_path
    ])
    
    torch.cuda.empty_cache()
    gc.collect()

finish = time.time()
print('time taken:', finish - start)

Input video path: /kaggle/input/neilafaw/video_0.mp4
Input audio path: /kaggle/input/neilafaw/audio_0.wav
Loaded checkpoint path: checkpoints/latentsync_unet.pt
Initial seed: 1247
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'prefer_nhwc': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_enable': '0', 'enable_cuda_graph': '0', 'fuse_conv_bias': '0', 'tunable_op_max_tuning_duration_ms': '0', 'tunable_op_tuning_enable': '0', 'cudnn_conv_use_max_workspace': '1', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_empty_cache': '0', 'has_user_compute_stream': '0', 'user_compute_stream': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'gpu_external_alloc': '0', 'sdpa_kernel': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'gpu_external_free': '0', 'use_tf32': '1', 'cudnn_conv1d_pad_to_nc1d': '0', 'do_copy_in_default_stream': '1'}}
m

Sample frames: 16:  90%|█████████ | 27/30 [00:26<00:02,  1.36it/s]